### Data scraped using wikitablescraper package via command line
##### Code used to scrape: python -m wikitablescrape --url="https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population" --output-folder="\test"
##### From resulting tables, table 5 was desired data. Data was renamed to my_data

In [112]:
import numpy as np
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import re
import requests

In [3]:
df = pd.read_csv('my_data.csv')

In [4]:
df

,2019 rank,City,State,2019 estimate,2010 Census,Change,2016 land area,2016 land area.1,2016 population density,2016 population density.1,Location
0,1,New York,New York,"8,336,817","8,175,133",+1.98%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...
1,2,Los Angeles,California,"3,979,576","3,792,621",+4.93%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...
2,3,Chicago,Illinois,"2,693,976","2,695,598",−0.06%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...
3,4,Houston,Texas,"2,320,268","2,100,263",+10.48%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...
4,5,Phoenix,Arizona,"1,680,992","1,445,632",+16.28%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...
...,...,...,...,...,...,...,...,...,...,...,...
312,313,San Angelo,Texas,"101,004","93,200",+8.37%,59.9 sq mi,155.1 km2,"1,681/sq mi",649/km2,31°26′28″N 100°27′02″W﻿ / ﻿31.4411°N 100.4505°...
313,314,Vacaville,California,"100,670","92,428",+8.92%,29.0 sq mi,75.1 km2,"3,449/sq mi","1,332/km2",38°21′14″N 121°58′22″W﻿ / ﻿38.3539°N 121.9728°...
314,315,Clinton,Michigan,"100,471","96,796",+3.80%,28.1 sq mi,72.8 km2,"3,573/sq mi","1,380/km2",42°35′25″N 82°55′01″W﻿ / ﻿42.5903°N 82.9170°W﻿...
315,316,Bend,Oregon,"100,421","76,639",+31.03%,33.1 sq mi,85.7 km2,"3,034/sq mi","1,171/km2",44°03′00″N 121°18′00″W﻿ / ﻿44.0500°N 121.3000°...


In [74]:
my_data = df[['City', 'State', 'Location']]

In [75]:
my_data

,City,State,Location
0,New York,New York,40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...
1,Los Angeles,California,34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...
2,Chicago,Illinois,41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...
3,Houston,Texas,29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...
4,Phoenix,Arizona,33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...
...,...,...,...
312,San Angelo,Texas,31°26′28″N 100°27′02″W﻿ / ﻿31.4411°N 100.4505°...
313,Vacaville,California,38°21′14″N 121°58′22″W﻿ / ﻿38.3539°N 121.9728°...
314,Clinton,Michigan,42°35′25″N 82°55′01″W﻿ / ﻿42.5903°N 82.9170°W﻿...
315,Bend,Oregon,44°03′00″N 121°18′00″W﻿ / ﻿44.0500°N 121.3000°...


In [96]:
def clean_location(row):
    split_data = row['Location'].split('/')[2]
    lat, long = re.findall("[-\d.]+", split_data)[0:2]
    return lat, long

In [97]:
my_data[['lat', 'long']] = my_data.apply(lambda row: pd.Series(clean_location(row)), axis = 1)

c:\users\rober\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [98]:
my_data

,City,State,Location,lat,long
0,New York,New York,40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...,40.6635,-73.9387
1,Los Angeles,California,34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...,34.0194,-118.4108
2,Chicago,Illinois,41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...,41.8376,-87.6818
3,Houston,Texas,29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...,29.7866,-95.3909
4,Phoenix,Arizona,33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...,33.5722,-112.0901
...,...,...,...,...,...
312,San Angelo,Texas,31°26′28″N 100°27′02″W﻿ / ﻿31.4411°N 100.4505°...,31.4411,-100.4505
313,Vacaville,California,38°21′14″N 121°58′22″W﻿ / ﻿38.3539°N 121.9728°...,38.3539,-121.9728
314,Clinton,Michigan,42°35′25″N 82°55′01″W﻿ / ﻿42.5903°N 82.9170°W﻿...,42.5903,-82.9170
315,Bend,Oregon,44°03′00″N 121°18′00″W﻿ / ﻿44.0500°N 121.3000°...,44.0500,-121.3000


In [100]:
my_data.drop(['Location'], axis = 1, inplace = True)

c:\users\rober\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [101]:
my_data

,City,State,lat,long
0,New York,New York,40.6635,-73.9387
1,Los Angeles,California,34.0194,-118.4108
2,Chicago,Illinois,41.8376,-87.6818
3,Houston,Texas,29.7866,-95.3909
4,Phoenix,Arizona,33.5722,-112.0901
...,...,...,...,...
312,San Angelo,Texas,31.4411,-100.4505
313,Vacaville,California,38.3539,-121.9728
314,Clinton,Michigan,42.5903,-82.9170
315,Bend,Oregon,44.0500,-121.3000


In [103]:
US_Cities = folium.Map(location = [42.877742, -97.380979], zoom_start = 3)

for lat, lng, city, state in zip(my_data['lat'], my_data['long'], my_data['City'], my_data['State']):
    label = '{}, {}'.format(city, state)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
    [lat, lng],
    radius = 5,
    popup = label,
    color = 'red',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.7,
    parse_html = False).add_to(US_Cities)
    
US_Cities

In [107]:
CLIENT_ID = '21ZQGP2WHAQYNH0C2HL4HOSV1UXDOT11OYCOEWOZZZC5BNWE'
CLIENT_SECRET = 'HNPWSAKM24YYDOE2OLF0WPE0QGSWTVNJIV4MWQLTFDJY52WN'
VERSION = '20200602'

In [160]:
def getNearbyVenues(cities, states, latitudes, longitudes, radius = 5000, limit = 100):
    venues_list = []
    missed_cities = []
    for city, state, lat, lng in zip(cities, states, latitudes, longitudes):
        # print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            limit)
    
        try:  
            results = requests.get(url).json()["response"]['groups'][0]['items']
    
            venues_list.append([(
                city,
                state,
                lat,
                lng,
                v['venue']['name'],
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
        
        except: 
            print('Something went wrong when processing:', city, state)
            missed_cities.append(city)
            
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City',
                            'State',
                            'City Latitude',
                            'City Longitude',
                            'Venue',
                            'Venue Latitude',
                            'Venue Longitude',
                            'Venue Category']

    return(nearby_venues, missed_cities)

In [161]:
def catch_missed_cities(missed_city_list):
    iter = 0
    while len(missed_city_list) > 0:
        if iter >= 10:
            print('Too many tries, check the city data')
            break
        missed_data = my_data[my_data['City'].isin(missed_city_list)]
        missed_venues, missed_again = getNearbyVenues(cities = missed_data['City'], states = missed_data['State'], latitudes = missed_data['lat'], longitudes = missed_data['long'])
        try:
            US_venues = pd.concat([US_venues, missed_venues])
            iter += 1
        except:
            iter += 1
            continue
    return US_venues

In [155]:
#Uncomment to rerun
#US_venues, missed_cities = getNearbyVenues(cities = my_data['City'], states = my_data['State'], latitudes = my_data['lat'], longitudes = my_data['long'])
#US_venues = catch_missed_cities(missed_cities)
US_venues

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,New York,40.6635,-73.9387,Empanada City,40.661631,-73.950436,Empanada Restaurant
1,New York,40.6635,-73.9387,Glou,40.662949,-73.953869,Tapas Restaurant
2,New York,40.6635,-73.9387,PLG Coffee House and Tavern,40.660007,-73.953362,Café
3,New York,40.6635,-73.9387,The Food Sermon,40.664588,-73.953735,Caribbean Restaurant
4,New York,40.6635,-73.9387,Colina Cuervo,40.671330,-73.950420,Café
...,...,...,...,...,...,...,...
95,Stamford,41.0799,-73.5460,CVS pharmacy,41.104887,-73.547908,Pharmacy
96,Stamford,41.0799,-73.5460,Olé Molé,41.106511,-73.548190,Mexican Restaurant
97,Stamford,41.0799,-73.5460,Dunkin',41.094378,-73.519542,Donut Shop
98,Stamford,41.0799,-73.5460,Angel Tips (Formerly Tahiti'a Spa),41.047814,-73.564241,Spa


In [115]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            32.7007,
            -97.1247,
            5000,
            500)
test = requests.get(url).json()

In [118]:
test1 = test["response"]['groups'][0]['items']

In [119]:
test_list = []
test_list.append([(
                'Arlington',
                32.7007,
                -97.1247,
                v['venue']['name'],
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in test1])

In [165]:
missed_cities = ['Arlington', 'Irving', 'Grand Rapids', 'Glendale', 'Fort Collins', \
                 'Corona', 'Orange', 'Stamford', 'Elizabeth', 'Thousand Oaks', 'Fargo', 'Antioch', 'Tuscaloosa', 'Vacaville', 'Woodbridge']
missed_cities

['Arlington',
 'Irving',
 'Grand Rapids',
 'Glendale',
 'Fort Collins',
 'Corona',
 'Orange',
 'Stamford',
 'Elizabeth',
 'Thousand Oaks',
 'Fargo',
 'Antioch',
 'Tuscaloosa',
 'Vacaville',
 'Woodbridge']

In [166]:
missed_data = my_data[my_data['City'].isin(missed_cities)]
missed_data

,City,State,lat,long
48,Arlington,Texas,32.7007,-97.1247
87,Glendale,Arizona,33.5331,-112.1899
94,Irving,Texas,32.8577,-96.9700
112,Grand Rapids,Michigan,42.9612,-85.6556
118,Glendale,California,34.1814,-118.2458
154,Fort Collins,Colorado,40.5482,-105.0648
155,Corona,California,33.8620,-117.5655
197,Orange,California,33.7870,-117.8613
213,Stamford,Connecticut,41.0799,-73.5460
215,Elizabeth,New Jersey,40.6664,-74.1935


In [135]:
missed_venues = getNearbyVenues(names = missed_data['City'], latitudes = missed_data['lat'], longitudes = missed_data['long'])
missed_venues

Something went wrong when processing: Stamford


,Neighborhood,Neighborhood Latitiude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Arlington,32.7007,-97.1247,Krispy Kreme Doughnuts,32.701612,-97.121649,Donut Shop
1,Arlington,32.7007,-97.1247,Texas Vision Care,32.697689,-97.127614,Optical Shop
2,Arlington,32.7007,-97.1247,Half Price Books,32.707720,-97.115157,Bookstore
3,Arlington,32.7007,-97.1247,Vandergriff Park,32.697561,-97.110060,Park
4,Arlington,32.7007,-97.1247,The Main Street Cafe,32.710842,-97.113418,Diner
...,...,...,...,...,...,...,...
1486,Woodbridge,40.5607,-74.2927,sri dosa palace,40.572316,-74.323796,Indian Restaurant
1487,Woodbridge,40.5607,-74.2927,Troy Restaurant,40.523276,-74.281075,Turkish Restaurant
1488,Woodbridge,40.5607,-74.2927,Artisan Bakery,40.538516,-74.241567,Bakery
1489,Woodbridge,40.5607,-74.2927,Ming,40.571027,-74.339365,Asian Restaurant


In [138]:
len(US_venues['Venue Latitude'].unique())

28429

In [167]:
#US_venues = pd.concat([US_venues, missed_venues])
US_venues

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,New York,40.6635,-73.9387,Empanada City,40.661631,-73.950436,Empanada Restaurant
1,New York,40.6635,-73.9387,Glou,40.662949,-73.953869,Tapas Restaurant
2,New York,40.6635,-73.9387,PLG Coffee House and Tavern,40.660007,-73.953362,Café
3,New York,40.6635,-73.9387,The Food Sermon,40.664588,-73.953735,Caribbean Restaurant
4,New York,40.6635,-73.9387,Colina Cuervo,40.671330,-73.950420,Café
...,...,...,...,...,...,...,...
95,Stamford,41.0799,-73.5460,CVS pharmacy,41.104887,-73.547908,Pharmacy
96,Stamford,41.0799,-73.5460,Olé Molé,41.106511,-73.548190,Mexican Restaurant
97,Stamford,41.0799,-73.5460,Dunkin',41.094378,-73.519542,Donut Shop
98,Stamford,41.0799,-73.5460,Angel Tips (Formerly Tahiti'a Spa),41.047814,-73.564241,Spa


In [141]:
missed_city = ['Stamford']
missed_city_data = my_data[my_data['City'].isin(missed_city)]
missed_city_venues = getNearbyVenues(names = missed_city_data['City'], latitudes = missed_city_data['lat'], longitudes = missed_city_data['long'])
missed_city_venues

,Neighborhood,Neighborhood Latitiude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Stamford,41.0799,-73.5460,Liz-Sue Bagels,41.073785,-73.548804,Bagel Shop
1,Stamford,41.0799,-73.5460,Coalhouse Pizza,41.074344,-73.548866,Pizza Place
2,Stamford,41.0799,-73.5460,Tabouli Grill,41.073730,-73.548542,Middle Eastern Restaurant
3,Stamford,41.0799,-73.5460,HomeGoods,41.072649,-73.548147,Furniture / Home Store
4,Stamford,41.0799,-73.5460,Great Nails,41.075948,-73.549287,Spa
...,...,...,...,...,...,...,...
95,Stamford,41.0799,-73.5460,CVS pharmacy,41.104887,-73.547908,Pharmacy
96,Stamford,41.0799,-73.5460,Olé Molé,41.106511,-73.548190,Mexican Restaurant
97,Stamford,41.0799,-73.5460,Dunkin',41.094378,-73.519542,Donut Shop
98,Stamford,41.0799,-73.5460,Angel Tips (Formerly Tahiti'a Spa),41.047814,-73.564241,Spa


In [142]:
US_venues = pd.concat([US_venues, missed_city_venues])
US_venues

,Neighborhood,Neighborhood Latitiude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,New York,40.6635,-73.9387,Empanada City,40.661631,-73.950436,Empanada Restaurant
1,New York,40.6635,-73.9387,Glou,40.662949,-73.953869,Tapas Restaurant
2,New York,40.6635,-73.9387,PLG Coffee House and Tavern,40.660007,-73.953362,Café
3,New York,40.6635,-73.9387,The Food Sermon,40.664588,-73.953735,Caribbean Restaurant
4,New York,40.6635,-73.9387,Colina Cuervo,40.671330,-73.950420,Café
...,...,...,...,...,...,...,...
95,Stamford,41.0799,-73.5460,CVS pharmacy,41.104887,-73.547908,Pharmacy
96,Stamford,41.0799,-73.5460,Olé Molé,41.106511,-73.548190,Mexican Restaurant
97,Stamford,41.0799,-73.5460,Dunkin',41.094378,-73.519542,Donut Shop
98,Stamford,41.0799,-73.5460,Angel Tips (Formerly Tahiti'a Spa),41.047814,-73.564241,Spa


In [164]:
#US_venues.rename(columns={'Neighborhood':'City', 'Neighborhood Latitiude':'City Latitude', 'Neighborhood Longitude':'City Longitude'}, inplace = True)
US_venues.groupby('City Latitude').count()

,City,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
City Latitude,,,,,,
21.3243,100,100,100,100,100,100
25.7752,100,100,100,100,100,100
25.8699,100,100,100,100,100,100
25.9489,100,100,100,100,100,100
25.9770,100,100,100,100,100,100
...,...,...,...,...,...,...
47.5979,100,100,100,100,100,100
47.6205,100,100,100,100,100,100
47.6669,89,89,89,89,89,89


In [148]:
print('We have {} unique categories'.format(len(US_venues['Venue Category'].unique())))

We have 493 unique categories


In [170]:
#my_data.rename(columns={'lat':'City Latitude', 'long':'City Longitude'}, inplace=True)
my_data

,City,State,City Latitude,City Longitude
0,New York,New York,40.6635,-73.9387
1,Los Angeles,California,34.0194,-118.4108
2,Chicago,Illinois,41.8376,-87.6818
3,Houston,Texas,29.7866,-95.3909
4,Phoenix,Arizona,33.5722,-112.0901
...,...,...,...,...
312,San Angelo,Texas,31.4411,-100.4505
313,Vacaville,California,38.3539,-121.9728
314,Clinton,Michigan,42.5903,-82.9170
315,Bend,Oregon,44.0500,-121.3000


In [171]:
corrected_venues = pd.merge(US_venues, my_data, how = 'left', on=['City', 'City Latitude', 'City Longitude'])
corrected_venues

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,State
0,New York,40.6635,-73.9387,Empanada City,40.661631,-73.950436,Empanada Restaurant,New York
1,New York,40.6635,-73.9387,Glou,40.662949,-73.953869,Tapas Restaurant,New York
2,New York,40.6635,-73.9387,PLG Coffee House and Tavern,40.660007,-73.953362,Café,New York
3,New York,40.6635,-73.9387,The Food Sermon,40.664588,-73.953735,Caribbean Restaurant,New York
4,New York,40.6635,-73.9387,Colina Cuervo,40.671330,-73.950420,Café,New York
...,...,...,...,...,...,...,...,...
30175,Stamford,41.0799,-73.5460,CVS pharmacy,41.104887,-73.547908,Pharmacy,Connecticut
30176,Stamford,41.0799,-73.5460,Olé Molé,41.106511,-73.548190,Mexican Restaurant,Connecticut
30177,Stamford,41.0799,-73.5460,Dunkin',41.094378,-73.519542,Donut Shop,Connecticut
30178,Stamford,41.0799,-73.5460,Angel Tips (Formerly Tahiti'a Spa),41.047814,-73.564241,Spa,Connecticut


In [176]:
#corrected_venues['City_State'] = corrected_venues['City'] + ", " + corrected_venues['State']
#corrected_venues.drop(['City', 'State'], axis = 1, inplace = True)
corrected_venues

,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,City_State
0,40.6635,-73.9387,Empanada City,40.661631,-73.950436,Empanada Restaurant,"New York, New York"
1,40.6635,-73.9387,Glou,40.662949,-73.953869,Tapas Restaurant,"New York, New York"
2,40.6635,-73.9387,PLG Coffee House and Tavern,40.660007,-73.953362,Café,"New York, New York"
3,40.6635,-73.9387,The Food Sermon,40.664588,-73.953735,Caribbean Restaurant,"New York, New York"
4,40.6635,-73.9387,Colina Cuervo,40.671330,-73.950420,Café,"New York, New York"
...,...,...,...,...,...,...,...
30175,41.0799,-73.5460,CVS pharmacy,41.104887,-73.547908,Pharmacy,"Stamford, Connecticut"
30176,41.0799,-73.5460,Olé Molé,41.106511,-73.548190,Mexican Restaurant,"Stamford, Connecticut"
30177,41.0799,-73.5460,Dunkin',41.094378,-73.519542,Donut Shop,"Stamford, Connecticut"
30178,41.0799,-73.5460,Angel Tips (Formerly Tahiti'a Spa),41.047814,-73.564241,Spa,"Stamford, Connecticut"


In [180]:
cities_onehot = pd.get_dummies(corrected_venues[['Venue Category']], prefix = '', prefix_sep = '')

cities_onehot['City_State'] = corrected_venues['City_State']

print(cities_onehot.columns.get_loc('City_State'))
onehot_location = cities_onehot.columns.get_loc('City_State')

fixed_columns = [cities_onehot.columns[onehot_location]] + list(cities_onehot.columns[:onehot_location]) + list(cities_onehot.columns[onehot_location+1:])
cities_onehot = cities_onehot[fixed_columns]

cities_onehot.head()

493


,City_State,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,...,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,"New York, New York",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"New York, New York",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"New York, New York",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"New York, New York",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"New York, New York",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [151]:
cities_onehot.shape

(30180, 494)

In [181]:
cities_grouped = cities_onehot.groupby('City_State').mean().reset_index()
cities_grouped

,City_State,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,...,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,"Abilene, Texas",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.01,0.000000,0.00,0.02,0.00,0.00,0.01,0.0
1,"Akron, Ohio",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.01,0.000000,0.00,0.00,0.00,0.00,0.02,0.0
2,"Albuquerque, New Mexico",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.000000,0.01,0.00,0.00,0.00,0.01,0.0
3,"Alexandria, Virginia",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.010000,0.00,0.00,0.00,0.01,0.00,0.0
4,"Allen, Texas",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.01,0.010000,0.00,0.01,0.01,0.00,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,"Wilmington, North Carolina",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.010101,0.00,0.00,0.00,0.00,0.00,0.0
312,"Winston–Salem, North Carolina",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.02,0.020000,0.00,0.00,0.01,0.01,0.00,0.0
313,"Woodbridge, New Jersey",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.020000,0.00,0.01,0.00,0.00,0.00,0.0
314,"Worcester, Massachusetts",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.01,0.01,0.010000,0.00,0.00,0.00,0.00,0.00,0.0


In [183]:
cities_grouped.shape

(316, 494)

In [188]:
#Skipping this since it would print a montrous amount of lines/data, uncomment if you'd like

top_venues = 5
'''
for group in cities_grouped['City_State']:
    print("----"+group+"----")
    temp = cities_grouped[cities_grouped['City_State'] == group].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(top_venues))
    print('\n')
'''

'\nfor group in cities_grouped[\'City_State\']:\n    print("----"+group+"----")\n    temp = cities_grouped[cities_grouped[\'City_State\'] == group].T.reset_index()\n    temp.columns = [\'venue\',\'freq\']\n    temp = temp.iloc[1:]\n    temp[\'freq\'] = temp[\'freq\'].astype(float)\n    temp = temp.round({\'freq\': 2})\n    print(temp.sort_values(\'freq\', ascending=False).reset_index(drop=True).head(top_venues))\n    print(\'\n\')\n'

In [194]:
def return_most_common_venues(row, top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending = False)
    
    return row_categories_sorted.index.values[0:top_venues]

In [196]:
top_venues = 20

indicators = ['st', 'nd', 'rd']

columns  = ['City_State']
            
for i in np.arange(top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(i+1, indicators[i]))
    except:
        columns.append('{}th Most Common Venue'.format(i+1))

city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['City_State'] = cities_grouped['City_State']

for ind in np.arange(cities_grouped.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(cities_grouped.iloc[ind, :], top_venues)

city_venues_sorted.head()

,City_State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,"Abilene, Texas",Mexican Restaurant,American Restaurant,Discount Store,Grocery Store,Fast Food Restaurant,Coffee Shop,Sandwich Place,Pharmacy,Convenience Store,...,Hotel,Donut Shop,Restaurant,Burger Joint,Pizza Place,Chinese Restaurant,BBQ Joint,Fried Chicken Joint,Bar,Wings Joint
1,"Akron, Ohio",Bar,American Restaurant,Sandwich Place,Mexican Restaurant,Pizza Place,Discount Store,Dive Bar,Coffee Shop,Chinese Restaurant,...,Café,Zoo,Indie Movie Theater,Middle Eastern Restaurant,Music Venue,Gay Bar,Ice Cream Shop,Performing Arts Venue,Brewery,Breakfast Spot
2,"Albuquerque, New Mexico",Brewery,Mexican Restaurant,American Restaurant,Coffee Shop,Hotel,Pizza Place,Café,Farmers Market,BBQ Joint,...,Science Museum,Sandwich Place,Theater,New American Restaurant,Design Studio,Beach,Gay Bar,Taco Place,Garden,Bike Rental / Bike Share
3,"Alexandria, Virginia",Bakery,Grocery Store,Park,Mexican Restaurant,Coffee Shop,BBQ Joint,Café,Pizza Place,American Restaurant,...,Burger Joint,Thai Restaurant,Food Truck,Trail,Ice Cream Shop,Arts & Crafts Store,Cheese Shop,Mediterranean Restaurant,Building,Movie Theater
4,"Allen, Texas",American Restaurant,Italian Restaurant,Pizza Place,Ice Cream Shop,Fast Food Restaurant,Mexican Restaurant,Coffee Shop,Grocery Store,BBQ Joint,...,Shopping Plaza,Discount Store,Asian Restaurant,Sushi Restaurant,Gym,Café,Supermarket,Burger Joint,Donut Shop,Hardware Store


In [197]:
k = 10

cities_grouped_clustering = cities_grouped.drop('City_State', 1)

kmeans = KMeans(n_clusters = k, random_state = 0).fit(cities_grouped_clustering)

kmeans.labels_[0:10]

array([6, 7, 7, 7, 1, 7, 6, 3, 7, 3])

In [199]:
my_data['City_State'] = my_data['City'] + ', ' + my_data['State']
my_data.drop(['City', 'State'], axis=1, inplace=True)
my_data

<ipython-input-199-1331fed5c0d8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_data['City_State'] = my_data['City'] + ', ' + my_data['State']
c:\users\rober\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,City Latitude,City Longitude,City_State
0,40.6635,-73.9387,"New York, New York"
1,34.0194,-118.4108,"Los Angeles, California"
2,41.8376,-87.6818,"Chicago, Illinois"
3,29.7866,-95.3909,"Houston, Texas"
4,33.5722,-112.0901,"Phoenix, Arizona"
...,...,...,...
312,31.4411,-100.4505,"San Angelo, Texas"
313,38.3539,-121.9728,"Vacaville, California"
314,42.5903,-82.9170,"Clinton, Michigan"
315,44.0500,-121.3000,"Bend, Oregon"


In [200]:
city_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

cities_merged = my_data

cities_merged = cities_merged.merge(city_venues_sorted, on = 'City_State')

cities_merged.shape

(316, 24)

In [201]:
clusters = folium.Map(location=[42.877742, -97.380979], zoom_start=4)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cities_merged['City Latitude'], cities_merged['City Longitude'], cities_merged['City_State'], cities_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=7,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(clusters)
       
clusters

In [205]:
cities_merged.loc[cities_merged['Cluster Labels'] == 0, cities_merged.columns[[2] + list(range(4, cities_merged.shape[1]))]]

,City_State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
58,"Corpus Christi, Texas",Resort,Beach,Residential Building (Apartment / Condo),Bay,American Restaurant,Zoo Exhibit,Event Space,Fabric Shop,Eye Doctor,...,Event Service,Fair,Ethiopian Restaurant,Entertainment Service,English Restaurant,Empanada Restaurant,Electronics Store,Factory,Farm,Falafel Restaurant


In [206]:
cities_merged.loc[cities_merged['Cluster Labels'] == 1, cities_merged.columns[[2] + list(range(4, cities_merged.shape[1]))]]

,City_State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
23,"Detroit, Michigan",Bar,Pizza Place,Middle Eastern Restaurant,American Restaurant,Fast Food Restaurant,Pharmacy,Bakery,Art Museum,History Museum,...,Grocery Store,Neighborhood,Gym,Performing Arts Venue,Deli / Bodega,Coffee Shop,Diner,Art Gallery,Café,Rock Club
34,"Mesa, Arizona",Coffee Shop,Pizza Place,Restaurant,Sushi Restaurant,Italian Restaurant,Convenience Store,Grocery Store,Bakery,Fast Food Restaurant,...,Sandwich Place,Burger Joint,Mexican Restaurant,Seafood Restaurant,Breakfast Spot,Thai Restaurant,Movie Theater,Asian Restaurant,Pet Store,Chinese Restaurant
39,"Omaha, Nebraska",American Restaurant,Grocery Store,Bar,Pizza Place,Park,Coffee Shop,Bakery,Pet Store,Fast Food Restaurant,...,Donut Shop,Thai Restaurant,Italian Restaurant,Mexican Restaurant,Furniture / Home Store,Café,Arts & Crafts Store,Shoe Store,Steakhouse,Shopping Mall
40,"Raleigh, North Carolina",Spa,American Restaurant,Hotel,Mexican Restaurant,Lingerie Store,Grocery Store,Breakfast Spot,Juice Bar,Sporting Goods Shop,...,Bookstore,Coffee Shop,Steakhouse,Toy / Game Store,Bakery,Japanese Restaurant,Brewery,Park,Café,Shopping Mall
43,"Virginia Beach, Virginia",Supermarket,Convenience Store,Brewery,Golf Course,Seafood Restaurant,Park,Campground,Mexican Restaurant,Italian Restaurant,...,Hardware Store,Pizza Place,Construction & Landscaping,Lingerie Store,Mini Golf,Liquor Store,Steakhouse,Big Box Store,Frozen Yogurt Shop,Thai Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,"South Bend, Indiana",Italian Restaurant,Grocery Store,Ice Cream Shop,Hotel,Mexican Restaurant,Deli / Bodega,Chinese Restaurant,Breakfast Spot,Bar,...,Greek Restaurant,Asian Restaurant,Farmers Market,BBQ Joint,Café,Coffee Shop,Gym,Discount Store,Liquor Store,Pizza Place
306,"Davenport, Iowa",American Restaurant,Grocery Store,Mexican Restaurant,Sandwich Place,Fast Food Restaurant,Coffee Shop,Pizza Place,Brewery,Italian Restaurant,...,Bar,Ice Cream Shop,Greek Restaurant,Dive Bar,Gym / Fitness Center,BBQ Joint,Breakfast Spot,Cosmetics Shop,Lingerie Store,Department Store
309,"Carmel, Indiana",Pizza Place,American Restaurant,Coffee Shop,Grocery Store,Mexican Restaurant,Liquor Store,Ice Cream Shop,Breakfast Spot,Burger Joint,...,Spa,Brewery,Deli / Bodega,Cajun / Creole Restaurant,Golf Course,Bar,Chinese Restaurant,Bakery,Steakhouse,Sporting Goods Shop
313,"Clinton, Michigan",Bar,Pizza Place,American Restaurant,Sandwich Place,Coffee Shop,Thai Restaurant,Grocery Store,Cosmetics Shop,Supermarket,...,Italian Restaurant,Bakery,Mexican Restaurant,General Entertainment,Convenience Store,Breakfast Spot,Pub,Restaurant,Burger Joint,Discount Store


In [207]:
cities_merged.loc[cities_merged['Cluster Labels'] == 2, cities_merged.columns[[2] + list(range(4, cities_merged.shape[1]))]]

,City_State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
15,"San Francisco, California",Outdoors & Recreation,Zoo Exhibit,Dumpling Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,...,Exhibit,Eye Doctor,Fabric Shop,Factory,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Festival


In [208]:
cities_merged.loc[cities_merged['Cluster Labels'] == 3, cities_merged.columns[[2] + list(range(4, cities_merged.shape[1]))]]

,City_State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
1,"Los Angeles, California",Grocery Store,Mexican Restaurant,Coffee Shop,Japanese Restaurant,New American Restaurant,Bakery,Taco Place,Café,Furniture / Home Store,...,Farmers Market,Pizza Place,Burger Joint,Gourmet Shop,Mediterranean Restaurant,Comic Shop,BBQ Joint,Wine Shop,Multiplex,Brazilian Restaurant
7,"San Diego, California",Japanese Restaurant,Sandwich Place,Bubble Tea Shop,Korean Restaurant,Coffee Shop,Noodle House,Sushi Restaurant,Mexican Restaurant,Grocery Store,...,Bakery,Fried Chicken Joint,Dessert Shop,Supermarket,Poke Place,Deli / Bodega,Ramen Restaurant,Fast Food Restaurant,Hawaiian Restaurant,Warehouse Store
9,"San Jose, California",Vietnamese Restaurant,Mexican Restaurant,Bubble Tea Shop,Fast Food Restaurant,Coffee Shop,Park,Gym / Fitness Center,Seafood Restaurant,Bakery,...,Steakhouse,Golf Course,Sushi Restaurant,Grocery Store,Fried Chicken Joint,Lingerie Store,Cajun / Creole Restaurant,Sporting Goods Shop,Spa,Frozen Yogurt Shop
18,"Denver, Colorado",Coffee Shop,Brewery,Taco Place,Mexican Restaurant,American Restaurant,Restaurant,Italian Restaurant,Burger Joint,Hardware Store,...,Pizza Place,Marijuana Dispensary,Hotel,Pool,Park,Cosmetics Shop,Sandwich Place,Wine Shop,Burrito Place,Food Truck
26,"Las Vegas, Nevada",Park,Fast Food Restaurant,Mexican Restaurant,Grocery Store,Breakfast Spot,American Restaurant,Sushi Restaurant,Coffee Shop,Italian Restaurant,...,Bar,Sandwich Place,Thai Restaurant,Salon / Barbershop,Gym,Café,Fried Chicken Joint,Tex-Mex Restaurant,Bakery,Bank
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,"Renton, Washington",Coffee Shop,Mexican Restaurant,Thai Restaurant,Sandwich Place,Italian Restaurant,Sushi Restaurant,Asian Restaurant,Sports Bar,Japanese Restaurant,...,Pub,Frozen Yogurt Shop,Fried Chicken Joint,Donut Shop,Gym,Bar,Park,Vietnamese Restaurant,Brewery,Chinese Restaurant
305,"Vista, California",Mexican Restaurant,Coffee Shop,Fast Food Restaurant,Pizza Place,Sandwich Place,Brewery,Italian Restaurant,Thai Restaurant,American Restaurant,...,Seafood Restaurant,Deli / Bodega,Grocery Store,Restaurant,Japanese Restaurant,Café,Fried Chicken Joint,Chinese Restaurant,Bakery,Breakfast Spot
308,"Tuscaloosa, Alabama",Mexican Restaurant,Sandwich Place,Bar,Coffee Shop,Burger Joint,Pizza Place,Fast Food Restaurant,Seafood Restaurant,Deli / Bodega,...,Brewery,Ice Cream Shop,BBQ Joint,Pub,Bakery,Asian Restaurant,Mediterranean Restaurant,Park,American Restaurant,Cosmetics Shop
310,"Spokane Valley, Washington",Coffee Shop,Sandwich Place,American Restaurant,Bar,Breakfast Spot,Burger Joint,Supermarket,Grocery Store,Mexican Restaurant,...,Ice Cream Shop,Beer Store,Chinese Restaurant,Thai Restaurant,Bakery,Restaurant,Asian Restaurant,Lingerie Store,Cosmetics Shop,Hobby Shop


In [209]:
cities_merged.loc[cities_merged['Cluster Labels'] == 4, cities_merged.columns[[2] + list(range(4, cities_merged.shape[1]))]]

,City_State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
220,"Norman, Oklahoma",Construction & Landscaping,Convenience Store,Electronics Store,Discount Store,Zoo Exhibit,Event Space,Factory,Fabric Shop,Eye Doctor,...,Ethiopian Restaurant,Event Service,Falafel Restaurant,Entertainment Service,English Restaurant,Empanada Restaurant,Fair,Farm,Dumpling Restaurant,Farmers Market


In [210]:
cities_merged.loc[cities_merged['Cluster Labels'] == 5, cities_merged.columns[[2] + list(range(4, cities_merged.shape[1]))]]

,City_State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
21,"El Paso, Texas",Convenience Store,Mexican Restaurant,Discount Store,Breakfast Spot,Pizza Place,Grocery Store,Fast Food Restaurant,Sandwich Place,Clothing Store,...,Ice Cream Shop,Fried Chicken Joint,Donut Shop,Steakhouse,Rental Car Location,Park,Burger Joint,Seafood Restaurant,Chinese Restaurant,Supplement Shop
32,"Tucson, Arizona",Convenience Store,Sandwich Place,Fast Food Restaurant,Pizza Place,Hotel,Grocery Store,Park,Coffee Shop,Gas Station,...,Breakfast Spot,Discount Store,Golf Course,Sports Bar,Bank,Government Building,Mexican Restaurant,Bar,Music Store,Burger Joint
49,"New Orleans, Louisiana",Fast Food Restaurant,Discount Store,Seafood Restaurant,Vietnamese Restaurant,Pharmacy,Bank,Gas Station,Big Box Store,Fried Chicken Joint,...,Grocery Store,Sandwich Place,Pizza Place,Gym / Fitness Center,Coffee Shop,Donut Shop,Breakfast Spot,Mexican Restaurant,Park,Rental Car Location
86,"Glendale, Arizona",Fast Food Restaurant,Convenience Store,Pizza Place,Sandwich Place,Mexican Restaurant,Pharmacy,Discount Store,Grocery Store,Park,...,Gas Station,Bank,Bar,Thai Restaurant,Restaurant,Video Game Store,Breakfast Spot,Candy Store,Golf Course,BBQ Joint
87,"North Las Vegas, Nevada",Mobile Phone Shop,Bar,Fast Food Restaurant,Coffee Shop,Pharmacy,Taco Place,Park,Convenience Store,Wings Joint,...,Sandwich Place,Gym,Gas Station,Pizza Place,Clothing Store,Chinese Restaurant,Food Truck,Fried Chicken Joint,Golf Course,Paper / Office Supplies Store
89,"Chesapeake, Virginia",Fast Food Restaurant,Gym,Convenience Store,Sandwich Place,Sushi Restaurant,Discount Store,Pharmacy,Park,American Restaurant,...,Italian Restaurant,Shoe Store,Grocery Store,Coffee Shop,Golf Course,Bar,Thrift / Vintage Store,Martial Arts Dojo,Business Service,Gas Station
100,"San Bernardino, California",Fast Food Restaurant,Sandwich Place,Mexican Restaurant,Pizza Place,Discount Store,Grocery Store,Coffee Shop,Bank,Bar,...,Convenience Store,Burger Joint,Breakfast Spot,Golf Course,Fried Chicken Joint,Gym,Department Store,Sushi Restaurant,Ice Cream Shop,Taco Place
104,"Moreno Valley, California",Fast Food Restaurant,Pizza Place,Pharmacy,Coffee Shop,Mexican Restaurant,Convenience Store,Chinese Restaurant,Sandwich Place,Thai Restaurant,...,Grocery Store,American Restaurant,Asian Restaurant,Gym,Furniture / Home Store,Department Store,Bank,Video Store,Steakhouse,Ice Cream Shop
121,"Augusta, Georgia",Discount Store,Pizza Place,Fast Food Restaurant,Convenience Store,Sandwich Place,Pharmacy,Gas Station,Grocery Store,Breakfast Spot,...,Fried Chicken Joint,Video Store,Automotive Shop,Gym,Skating Rink,General Entertainment,Construction & Landscaping,Moving Target,Factory,Electronics Store
126,"Columbus, Georgia",Fast Food Restaurant,Discount Store,Pizza Place,Convenience Store,Video Store,Grocery Store,Breakfast Spot,Sandwich Place,Chinese Restaurant,...,Fried Chicken Joint,Mexican Restaurant,Park,BBQ Joint,Shipping Store,Liquor Store,Bagel Shop,Bar,Baseball Field,Gas Station


In [211]:
cities_merged.loc[cities_merged['Cluster Labels'] == 6, cities_merged.columns[[2] + list(range(4, cities_merged.shape[1]))]]

,City_State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
2,"Chicago, Illinois",Mexican Restaurant,Coffee Shop,Park,Bar,Diner,Taco Place,Italian Restaurant,Dessert Shop,Hot Dog Joint,...,Café,Art Gallery,Grocery Store,Ice Cream Shop,Brewery,New American Restaurant,Vietnamese Restaurant,Cocktail Bar,Latin American Restaurant,Liquor Store
4,"Phoenix, Arizona",Mexican Restaurant,Fast Food Restaurant,Convenience Store,Coffee Shop,Trail,Italian Restaurant,Hotel,Burger Joint,Pizza Place,...,Thrift / Vintage Store,BBQ Joint,Bookstore,Ice Cream Shop,Greek Restaurant,Grocery Store,Video Store,American Restaurant,Scenic Lookout,Pharmacy
6,"San Antonio, Texas",Burger Joint,Mexican Restaurant,Coffee Shop,Bar,Grocery Store,Park,Pizza Place,American Restaurant,BBQ Joint,...,Gay Bar,Big Box Store,Taco Place,Breakfast Spot,Restaurant,Clothing Store,College Auditorium,Recreation Center,Donut Shop,Furniture / Home Store
12,"Fort Worth, Texas",Mexican Restaurant,Bar,Burger Joint,Steakhouse,Hotel,Beer Bar,Art Museum,New American Restaurant,Brewery,...,Nightclub,Café,Shopping Plaza,Park,Lounge,Sushi Restaurant,Historic Site,Fast Food Restaurant,Brazilian Restaurant,Breakfast Spot
28,"Louisville, Kentucky",Mexican Restaurant,Sandwich Place,Fast Food Restaurant,Discount Store,Pharmacy,American Restaurant,Pet Store,Coffee Shop,Italian Restaurant,...,BBQ Joint,Chinese Restaurant,Clothing Store,Seafood Restaurant,Rental Car Location,Warehouse Store,Bakery,Liquor Store,Bar,Lingerie Store
42,"Long Beach, California",Mexican Restaurant,Fast Food Restaurant,Diner,Sushi Restaurant,Discount Store,Grocery Store,Thai Restaurant,Park,American Restaurant,...,Brewery,Gym,Seafood Restaurant,Pizza Place,Food Court,Japanese Restaurant,Italian Restaurant,Rental Car Location,Hotel,Restaurant
46,"Tulsa, Oklahoma",Burger Joint,Mexican Restaurant,Convenience Store,Ice Cream Shop,Pizza Place,Vietnamese Restaurant,Fried Chicken Joint,Diner,Sandwich Place,...,Greek Restaurant,BBQ Joint,Bakery,Bar,Coffee Shop,American Restaurant,Hotel,Cosmetics Shop,Jewelry Store,Pharmacy
51,"Bakersfield, California",Mexican Restaurant,Pizza Place,Fast Food Restaurant,Sandwich Place,Coffee Shop,Pharmacy,Burger Joint,American Restaurant,Clothing Store,...,Lingerie Store,Music Store,Mediterranean Restaurant,Rental Car Location,Discount Store,Department Store,Convenience Store,Cosmetics Shop,Asian Restaurant,Big Box Store
73,"Chula Vista, California",Mexican Restaurant,Park,Pharmacy,Grocery Store,Coffee Shop,Chinese Restaurant,Supermarket,Asian Restaurant,Seafood Restaurant,...,Taco Place,Liquor Store,Pet Store,Nail Salon,Sushi Restaurant,Playground,Burger Joint,Pizza Place,Cosmetics Shop,Fast Food Restaurant
77,"Laredo, Texas",Mexican Restaurant,Department Store,Clothing Store,Burger Joint,Hotel,Seafood Restaurant,Fast Food Restaurant,Steakhouse,Pharmacy,...,Lingerie Store,Frozen Yogurt Shop,Movie Theater,Ice Cream Shop,Pizza Place,Italian Restaurant,Wings Joint,Coffee Shop,Grocery Store,Furniture / Home Store


In [212]:
cities_merged.loc[cities_merged['Cluster Labels'] == 7, cities_merged.columns[[2] + list(range(4, cities_merged.shape[1]))]]

,City_State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,"New York, New York",Bar,Caribbean Restaurant,Cocktail Bar,Bakery,Café,Park,Pizza Place,New American Restaurant,Wine Shop,...,Beer Bar,Mexican Restaurant,Field,Ice Cream Shop,Sushi Restaurant,Restaurant,Gourmet Shop,Yoga Studio,Taco Place,Tapas Restaurant
3,"Houston, Texas",Coffee Shop,Trail,Mexican Restaurant,Taco Place,Cajun / Creole Restaurant,Yoga Studio,Grocery Store,Park,Gym,...,Food Truck,Cocktail Bar,Restaurant,BBQ Joint,Beer Garden,New American Restaurant,Bakery,Gastropub,Dessert Shop,Donut Shop
5,"Philadelphia, Pennsylvania",Bar,Café,Pizza Place,Caribbean Restaurant,Brewery,Bakery,Discount Store,Market,Athletics & Sports,...,Coffee Shop,Southern / Soul Food Restaurant,Fried Chicken Joint,Beer Garden,American Restaurant,Sandwich Place,Korean Restaurant,Convenience Store,Donut Shop,Seafood Restaurant
8,"Dallas, Texas",Bar,Burger Joint,Coffee Shop,American Restaurant,Park,Grocery Store,Pizza Place,Thai Restaurant,Dive Bar,...,Brewery,Gym / Fitness Center,Art Museum,Cocktail Bar,Bakery,Theater,Vietnamese Restaurant,Performing Arts Venue,Mexican Restaurant,Sports Bar
10,"Austin, Texas",Park,Ice Cream Shop,American Restaurant,Mexican Restaurant,Yoga Studio,Grocery Store,Bakery,Coffee Shop,Pizza Place,...,Burger Joint,Juice Bar,Capitol Building,History Museum,Café,Flower Shop,Bar,Food Truck,BBQ Joint,Taco Place
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,"Lansing, Michigan",Bar,Pizza Place,Coffee Shop,Mexican Restaurant,Convenience Store,Breakfast Spot,Brewery,Chinese Restaurant,BBQ Joint,...,Trail,Ice Cream Shop,Sandwich Place,Restaurant,Italian Restaurant,Candy Store,Middle Eastern Restaurant,Diner,Furniture / Home Store,Bakery
250,"Carlsbad, California",Hotel,American Restaurant,Brewery,Theme Park,Coffee Shop,Golf Course,Bakery,Deli / Bodega,Mexican Restaurant,...,Grocery Store,Gym / Fitness Center,Theme Park Ride / Attraction,Pizza Place,Park,Pool,Burger Joint,Sandwich Place,Farmers Market,Spa
261,"Lakeland, Florida",American Restaurant,BBQ Joint,Coffee Shop,Food Truck,Bar,Park,Cosmetics Shop,Sandwich Place,Breakfast Spot,...,Steakhouse,Fried Chicken Joint,Museum,Gym,Clothing Store,Bakery,Burger Joint,Sporting Goods Shop,Mexican Restaurant,Convenience Store
295,"Green Bay, Wisconsin",Bar,Mexican Restaurant,American Restaurant,Pizza Place,Brewery,Trail,Coffee Shop,Burger Joint,Thai Restaurant,...,Hotel,Gym / Fitness Center,Steakhouse,Sandwich Place,Smoke Shop,Convenience Store,Park,Farmers Market,Cocktail Bar,Theater


In [213]:
cities_merged.loc[cities_merged['Cluster Labels'] == 8, cities_merged.columns[[2] + list(range(4, cities_merged.shape[1]))]]

,City_State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
145,"Peoria, Arizona",Park,Golf Course,Pool,Athletics & Sports,Recreation Center,Grocery Store,Racetrack,Paintball Field,Drugstore,...,Event Space,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Driving School,Ethiopian Restaurant,Event Service,Zoo Exhibit


In [214]:
cities_merged.loc[cities_merged['Cluster Labels'] == 9, cities_merged.columns[[2] + list(range(4, cities_merged.shape[1]))]]

,City_State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
139,"Brownsville, Texas",Bar,Breakfast Spot,Monument / Landmark,Wine Bar,Restaurant,Music Venue,Zoo Exhibit,Event Space,Factory,...,Eye Doctor,Exhibit,Ethiopian Restaurant,Event Service,Falafel Restaurant,Entertainment Service,English Restaurant,Empanada Restaurant,Fair,Fast Food Restaurant
